In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skfda.representation.grid import FDataGrid
from skfda.representation.basis import FourierBasis
from skfda.preprocessing.smoothing import BasisSmoother
import glob
import re 
from skfda.misc.metrics import l2_distance

## Loading All Data

In [10]:
# Get all csv files in the folder
folder_path = 'Aligned Data All Participants'

# Initialize separate dictionaries
ax_series = []
la_series = []
fl_series = []

# Get all CSV files
all_files = glob.glob(os.path.join(folder_path, "*.csv"))

def FDtuple(df_imu, df_mcp):
    if not df_imu.empty and not df_mcp.empty:
        time_imu = np.round(df_imu.iloc[:, 0].values, 3)
        meas_imu = df_imu.iloc[:, 1].values
        time_mcp = np.round(df_mcp.iloc[:, 0].values, 3) 
        meas_mcp = df_mcp.iloc[:, 1].values

        t_min = min(time_imu.min(), time_mcp.min())
        t_max = max(time_imu.max(), time_mcp.max())
        domain = (t_min, t_max)

        return tuple([
            FDataGrid([meas_imu], grid_points=[time_imu], domain_range=domain),
            FDataGrid([meas_mcp], grid_points=[time_mcp], domain_range=domain)
        ])
    else:
        return None


for i in range(1, 31): 

    ax_imu = pd.DataFrame()
    ax_mcp = pd.DataFrame()
    la_imu = pd.DataFrame()
    la_mcp = pd.DataFrame()
    fl_imu = pd.DataFrame()
    fl_mcp = pd.DataFrame()

    for file in all_files:
        
        if re.search(rf'\\{i}(?!\d)', file):  
            if 'Axial' in file and 'IMU' in file: 
                ax_imu = pd.read_csv(file)
            elif 'Axial' in file and 'MoCap' in file:
                ax_mcp = pd.read_csv(file)
            elif 'Lateral' in file and 'IMU' in file:
                la_imu = pd.read_csv(file)
            elif 'Lateral' in file and 'MoCap' in file:
                la_mcp = pd.read_csv(file)
            elif 'Flexion' in file and 'IMU' in file:
                fl_imu = pd.read_csv(file)
            elif 'Flexion' in file and 'MoCap' in file:
                fl_mcp = pd.read_csv(file)
            else: 
                print('Warning: {file} isnt classified')
    
    if (ax := FDtuple(ax_imu, ax_mcp)):
        ax_series.append(ax)

    if (la := FDtuple(la_imu, la_mcp)):
        la_series.append(la)

    if (fl := FDtuple(fl_imu, fl_mcp)):
        fl_series.append(fl)

print(f"Loaded {len(ax_series)} AxialRotation participants")
print(f"Loaded {len(la_series)} Flexion participants")
print(f"Loaded {len(fl_series)} LateralBending participants")

Loaded 26 AxialRotation participants
Loaded 27 Flexion participants
Loaded 28 LateralBending participants


## Distances in Axial Rotations + Statistical Test

In [3]:
# Find global min and max time for all series
all_times = np.concatenate([fd.grid_points[0] for pair in ax_series for fd in pair])
t_min, t_max = np.min(all_times), np.max(all_times)

# Normalize all FDataGrids to time range [0, 1]
normalized_series = []

for s1, s2 in ax_series:
    norm_pair = []
    for fd in [s1, s2]:
        t = fd.grid_points[0]
        t_norm = (t - t_min) / (t_max - t_min)
        fd_norm = FDataGrid(data_matrix=fd.data_matrix, grid_points=[t_norm])
        norm_pair.append(fd_norm)
    normalized_series.append(tuple(norm_pair))

In [11]:
all_times = np.concatenate([fd.grid_points[0] for pair in ax_series for fd in pair])
t_min, t_max = np.min(all_times), np.max(all_times)
basis = FourierBasis(domain_range=(t_min, t_max), n_basis=7)
smoother = BasisSmoother(basis=basis)

def smooth_fd(fd: FDataGrid, t_min, t_max):
    # Use slightly inside [0, 1] range to avoid edge issues
    grid = np.linspace(t_min, t_max, 100000)
    fd_grid = fd.to_grid(grid)
    return smoother.fit_transform(fd_grid)

smoothed_pairs = []
for s1, s2 in ax_series:
    smoothed_s1 = smooth_fd(s1, t_min, t_max)
    smoothed_s2 = smooth_fd(s2, t_min, t_max)
    smoothed_pairs.append((smoothed_s1, smoothed_s2))

ValueError: Grid points must be within the domain range.

In [ ]:
# Compute L2 distance for each participant's two smoothed series
distances = []
for s1, s2 in smoothed_pairs:
    d = l2_distance(s1, s2)
    distances.append(d)

# Visualize distances
plt.hist(distances, bins=10)
plt.title("L2 Distances Between Paired Time Series")
plt.xlabel("Distance")
plt.ylabel("Count")
plt.show()

Aligned Data All Participants\10AxialRotation_IMU_95s_to_127s.csv
Aligned Data All Participants\10AxialRotation_MoCap_95s_to_127s.csv
Aligned Data All Participants\10Flexion_IMU_3s_to_44s.csv
Aligned Data All Participants\10Flexion_MoCap_3s_to_44s.csv
Aligned Data All Participants\10LateralBending_IMU_53s_to_85s.csv
Aligned Data All Participants\10LateralBending_MoCap_53s_to_85s.csv
Aligned Data All Participants\11AxialRotation_IMU_95s_to_127s.csv
Aligned Data All Participants\11AxialRotation_MoCap_95s_to_127s.csv
Aligned Data All Participants\11Flexion_IMU_3s_to_44s.csv
Aligned Data All Participants\11Flexion_MoCap_3s_to_44s.csv
Aligned Data All Participants\11LateralBending_IMU_53s_to_85s.csv
Aligned Data All Participants\11LateralBending_MoCap_53s_to_85s.csv
Aligned Data All Participants\12AxialRotation_IMU_95s_to_126s.csv
Aligned Data All Participants\12AxialRotation_MoCap_95s_to_126s.csv
Aligned Data All Participants\12Flexion_IMU_3s_to_44s.csv
Aligned Data All Participants\12Flex